# Finetuning using Axolotl and Python

This notebook is an minimal example of how to finetune a LLM using Axolotl and Python. Axolotl is a CLI tool that uses config files for different methods of LLM finetuning. We created a Python wrapper around the CLI for the end-to-end workflow for this process.

In the example below, we show how you can define or load finetuning configurations, start an finetuning job, and push it to Hugging Face.

## Setup

Make sure to run this notebook in a system with enough compute resources to run the finetuning, and follow the setup instructions in the README to install axolotl and related libraries.

Let's start with loading the code components we need. The `FinetuneConfig` class holds configurations, and the `Finetune` class is used to create and run a finetuning job.

In [3]:
import sys
import os

sys.path.insert(0, os.path.join(os.getcwd(), '..'))

from finetune import Finetune, FinetuneConfig

Also make sure to login to Hugging Face Hub to save the output model.

In [2]:
from huggingface_hub import login
login()

## Configuration
We then load up a config to perform QLoRA finetuning of Llama 2 from in a config file stored locally. Optionally, we can assign a new field `hub_model_id`, indicating the Hugging Face model the finetuned LLM will be pushed to.

In [6]:
import yaml

# Specify the path to your YAML file
file_path = os.path.join(os.getcwd(), '..', 'finetune/examples/llama-2/qlora.yml')

# Open the file and load the data
with open(file_path, encoding='utf-8') as file:
    config_dict = yaml.safe_load(file)  # Load the existing data

config_dict['hub_model_id'] = 'vijil/my_lora_tune'  # Add or update the model_id to push the trained model
config_dict['eval_sample_packing'] = False

In [7]:
config_dict

{'base_model': 'NousResearch/Llama-2-7b-hf',
 'model_type': 'LlamaForCausalLM',
 'tokenizer_type': 'LlamaTokenizer',
 'load_in_8bit': False,
 'load_in_4bit': True,
 'strict': False,
 'datasets': [{'path': 'mhenrichsen/alpaca_2k_test', 'type': 'alpaca'}],
 'dataset_prepared_path': None,
 'val_set_size': 0.05,
 'output_dir': './qlora-out',
 'adapter': 'qlora',
 'lora_model_dir': None,
 'sequence_len': 4096,
 'sample_packing': True,
 'pad_to_sequence_len': True,
 'lora_r': 32,
 'lora_alpha': 16,
 'lora_dropout': 0.05,
 'lora_target_modules': None,
 'lora_target_linear': True,
 'lora_fan_in_fan_out': None,
 'wandb_project': None,
 'wandb_entity': None,
 'wandb_watch': None,
 'wandb_name': None,
 'wandb_log_model': None,
 'gradient_accumulation_steps': 4,
 'micro_batch_size': 2,
 'num_epochs': 4,
 'optimizer': 'paged_adamw_32bit',
 'lr_scheduler': 'cosine',
 'learning_rate': 0.0002,
 'train_on_inputs': False,
 'group_by_length': False,
 'bf16': 'auto',
 'fp16': None,
 'tf32': False,
 'gradi

Let's now load the config dict in the `FinetuneConfig` object.

In [8]:
# see all config options in './finetune/examples/config.qmd'
config = FinetuneConfig(config_dict)

[2024-05-03 18:16:02,711] [DEBUG] [axolotl.normalize_config:79] [PID:18933] [RANK:0] bf16 support detected, enabling for this configuration.


/home/ec2-user/anaconda3/envs/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2024-05-03 18:16:02,835] [INFO] [axolotl.normalize_config:182] [PID:18933] [RANK:0] GPU memory usage baseline: 0.000GB (+0.423GB misc)


## Run the finetuning job
Now simply load up the config into a `FineTune` object and kick off the job.

In [9]:
# create a finetune object with the config and run
finetune = Finetune(config)

In [ ]:
finetune.run() # start train

[INFO] Job ID: 385293fa-0979-11ef-b980-024398e8947b
[2024-05-03 18:16:11,511] [DEBUG] [axolotl.load_tokenizer:279] [PID:18933] [RANK:0] EOS: 2 / </s>
[2024-05-03 18:16:11,512] [DEBUG] [axolotl.load_tokenizer:280] [PID:18933] [RANK:0] BOS: 1 / <s>
[2024-05-03 18:16:11,512] [DEBUG] [axolotl.load_tokenizer:281] [PID:18933] [RANK:0] PAD: 0 / <unk>
[2024-05-03 18:16:11,513] [DEBUG] [axolotl.load_tokenizer:282] [PID:18933] [RANK:0] UNK: 0 / <unk>
[2024-05-03 18:16:11,513] [INFO] [axolotl.load_tokenizer:293] [PID:18933] [RANK:0] No Chat template selected. Consider adding a chat template for easier inference.
[2024-05-03 18:16:11,514] [INFO] [axolotl.load_tokenized_prepared_datasets:183] [PID:18933] [RANK:0] Unable to find prepared dataset in last_run_prepared/5c09ab14f30fc60ead5860bcbbb2e263
[2024-05-03 18:16:11,514] [INFO] [axolotl.load_tokenized_prepared_datasets:184] [PID:18933] [RANK:0] Loading raw datasets...
[2024-05-03 18:16:11,515] [WARNING] [axolotl.load_tokenized_prepared_datasets:1

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

[2024-05-03 18:16:18,626] [DEBUG] [axolotl.log:61] [PID:18933] [RANK:0] total_num_tokens: 414_041
[2024-05-03 18:16:18,666] [DEBUG] [axolotl.log:61] [PID:18933] [RANK:0] `total_supervised_tokens: 294_246`
[2024-05-03 18:16:22,644] [INFO] [axolotl.utils.samplers.multipack._len_est:184] [PID:18933] [RANK:0] packing_efficiency_estimate: 1.0 total_num_tokens per device: 414041
[2024-05-03 18:16:22,644] [DEBUG] [axolotl.log:61] [PID:18933] [RANK:0] data_loader_len: 6
[2024-05-03 18:16:22,645] [INFO] [axolotl.log:61] [PID:18933] [RANK:0] sample_packing_eff_est across ranks: [0.9719637357271634]
[2024-05-03 18:16:22,645] [DEBUG] [axolotl.log:61] [PID:18933] [RANK:0] sample_packing_eff_est: 0.98
[2024-05-03 18:16:22,646] [DEBUG] [axolotl.log:61] [PID:18933] [RANK:0] total_num_steps: 24
[2024-05-03 18:16:22,656] [DEBUG] [axolotl.train.log:61] [PID:18933] [RANK:0] loading tokenizer... NousResearch/Llama-2-7b-hf


/home/ec2-user/anaconda3/envs/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2024-05-03 18:16:22,908] [DEBUG] [axolotl.load_tokenizer:279] [PID:18933] [RANK:0] EOS: 2 / </s>
[2024-05-03 18:16:22,909] [DEBUG] [axolotl.load_tokenizer:280] [PID:18933] [RANK:0] BOS: 1 / <s>
[2024-05-03 18:16:22,909] [DEBUG] [axolotl.load_tokenizer:281] [PID:18933] [RANK:0] PAD: 0 / <unk>
[2024-05-03 18:16:22,910] [DEBUG] [axolotl.load_tokenizer:282] [PID:18933] [RANK:0] UNK: 0 / <unk>
[2024-05-03 18:16:22,910] [INFO] [axolotl.load_tokenizer:293] [PID:18933] [RANK:0] No Chat template selected. Consider adding a chat template for easier inference.
[2024-05-03 18:16:22,911] [DEBUG] [axolotl.train.log:61] [PID:18933] [RANK:0] loading model and peft_config...
[2024-05-03 18:16:23,278] [INFO] [axolotl.load_model:359] [PID:18933] [RANK:0] patching with flash attention for sample packing
[2024-05-03 18:16:23,279] [INFO] [axolotl.load_model:408] [PID:18933] [RANK:0] patching _expand_mask
[2024-05-03 18:16:23,604] [INFO] [accelerate.utils.modeling.get_balanced_memory:965] [PID:18933] We wil

...

[2024-05-03 19:08:27,541] [INFO] [accelerate.accelerator.log:61] [PID:18933] The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
[2024-05-03 19:08:27,645] [INFO] [axolotl.utils.samplers.multipack._len_est:184] [PID:18933] [RANK:0] packing_efficiency_estimate: 0.98 total_num_tokens per device: 414041
[2024-05-03 19:12:10,016] [INFO] [accelerate.accelerator.log:61] [PID:18933] The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
[2024-05-03 19:15:52,487] [INFO] [accelerate.accelerator.log:61] [PID:18933] The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
[2024-05-03 19:19:34,980] [INFO] [accelerate.accelerator.log:61] [PID:18933] The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.


/home/ec2-user/anaconda3/envs/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2024-05-03 19:19:44,329] [INFO] [axolotl.train.log:61] [PID:18933] [RANK:0] Training Completed!!! Saving pre-trained model to ./qlora-out


/home/ec2-user/anaconda3/envs/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/160M [00:00<?, ?B/s]

## Status
To check the job status, please use `finetune_axolotl-status.ipynb`.